In [27]:
## Dependencies

import pandas as pd
import pathlib as path


import requests
import json
from pprint import pprint
from bs4 import BeautifulSoup

import numpy as np

from io import StringIO

# to read the ZIP file
import requests
import zipfile
import io



# (1) create DF for countries

import from csv file 

In [33]:

path = '../Resources_Output/countries_UN_referential.csv'
countries_wb_df = pd.read_csv(path)

countries_wb_df.head()

,iso3Code,iso2Code,country,region,capitalCity,longitude,latitude
0,ABW,AW,Aruba,Latin America & Caribbean,Oranjestad,-70.0167,12.51670
1,AFG,AF,Afghanistan,South Asia,Kabul,69.1761,34.52280
2,AGO,AO,Angola,Sub-Saharan Africa,Luanda,13.2420,-8.81155
3,ALB,AL,Albania,Europe & Central Asia,Tirane,19.8172,41.33170
4,AND,AD,Andorra,Europe & Central Asia,Andorra la Vella,1.5218,42.50750


# (2) Extract Corruption perception indicator


source : https://images.transparencycdn.org/images/CPI2023_FullDataSet.zip

from webpage : https://www.transparency.org/en/cpi/2023/media-kit

extract sheet : CPI 2023

will be saved in directory : Resources_external

In [19]:
url = "https://images.transparencycdn.org/images/CPI2023_FullDataSet.zip"

# Download the ZIP file (code from Chat GPT)

response = requests.get(url)
if response.status_code == 200:

    # Step 2: Open the ZIP file from memory
    with zipfile.ZipFile(io.BytesIO(response.content), "r") as zip_ref:
        # Extract all files to a folder
        zip_ref.extractall("../Resources_external")
        print("Files extracted successfully!")

        # Step 3: Identify the Excel file (assuming there's only one)
        excel_file = [f for f in zip_ref.namelist() if f.endswith(".xlsx")][0]
        excel_path = f"../Resources_external/{excel_file}"

        # Step 4: Read the "CPI 2023" sheet into a DataFrame and starting from row 4
        cpi_df = pd.read_excel(excel_path, sheet_name="CPI 2023", skiprows=3)
        print("Sheet 'CPI 2023' loaded into a DataFrame successfully!")

        # Display the first few rows
        cpi_df.head()
else:
    print("Failed to download the file. Check the URL.")


Files extracted successfully!
Sheet 'CPI 2023' loaded into a DataFrame successfully!


In [20]:
cpi_df.head()

,Country / Territory,ISO3,Region,CPI score 2023,Rank,Standard error,Number of sources,Lower CI,Upper CI,African Development Bank CPIA,...,Economist Intelligence Unit Country Ratings,Freedom House Nations in Transit,Global Insights Country Risk Ratings,IMD World Competitiveness Yearbook,PERC Asia Risk Guide,PRS International Country Risk Guide,Varieties of Democracy Project,World Bank CPIA,World Economic Forum EOS,World Justice Project Rule of Law Index
0,Denmark,DNK,WE/EU,90,1,1.819265,8,87.01640,92.98360,NaN,...,89.67447,NaN,83.25637,97.89119,NaN,100.00000,78.89152,NaN,82.20667,87.68097
1,Finland,FIN,WE/EU,87,2,1.057962,8,85.26495,88.73505,NaN,...,89.67447,NaN,83.25637,89.84466,NaN,93.38018,78.16811,NaN,86.76612,85.88441
2,New Zealand,NZL,AP,85,3,1.916107,8,81.85758,88.14242,NaN,...,89.67447,NaN,83.25637,87.10542,NaN,93.38018,78.16811,NaN,70.98344,82.29130
3,Norway,NOR,WE/EU,84,4,1.380803,7,81.73548,86.26452,NaN,...,89.67447,NaN,83.25637,75.72045,NaN,84.67139,78.16811,NaN,NaN,86.78268
4,Singapore,SGP,AP,83,5,1.145734,9,81.12099,84.87901,NaN,...,89.67447,NaN,83.25637,76.74767,89.29896,83.27798,78.16811,NaN,88.51975,84.08785


In [21]:
cpi_df.columns

Index(['Country / Territory', 'ISO3', 'Region', 'CPI score 2023', 'Rank',
       'Standard error', 'Number of sources', 'Lower CI', 'Upper CI',
       'African Development Bank CPIA',
       'Bertelsmann Foundation Sustainable Governance Index',
       'Bertelsmann Foundation Transformation Index',
       'Economist Intelligence Unit Country Ratings',
       'Freedom House Nations in Transit',
       'Global Insights Country Risk Ratings',
       'IMD World Competitiveness Yearbook', 'PERC Asia Risk Guide',
       'PRS International Country Risk Guide',
       'Varieties of Democracy Project', 'World Bank CPIA',
       'World Economic Forum EOS', 'World Justice Project Rule of Law Index'],
      dtype='object')

In [22]:
cpi_df = cpi_df.rename(columns={
                                'Country / Territory' : 'country' , 
                                'ISO3' : 'iso3Code' , 
                                'CPI score 2023' : 'CPI_score_2023'
                                
                                })

cpi_df = cpi_df[['country' , 'iso3Code', 'CPI_score_2023']]

cpi_df.columns

Index(['country', 'iso3Code', 'CPI_score_2023'], dtype='object')

In [23]:
cpi_df.shape

(180, 3)

In [24]:
cpi_df.head(5)

,country,iso3Code,CPI_score_2023
0,Denmark,DNK,90
1,Finland,FIN,87
2,New Zealand,NZL,85
3,Norway,NOR,84
4,Singapore,SGP,83


In [25]:
# Save to CSV 
cpi_df.to_csv("../Resources_Output/corruption_perception_index.csv", index=False)

# (3) Extract Global_Peace_Index data
source Wikidepia : https://en.wikipedia.org/wiki/Global_Peace_Index

the full data and the report is available as PDF from the Economics And Peace organization
link : https://www.economicsandpeace.org/wp-content/uploads/2023/09/GPI-2023-Web.pdf

In [10]:
# extract the data from Wikipedia page

## define URL
url = "https://en.wikipedia.org/wiki/Global_Peace_Index"

# Read all tables from the page
tables = pd.read_html(url)

# Find the correct table
gpi_df = tables[1]  

# Display the extracted table
gpi_df = gpi_df.sort_values('Country', ascending=True)
gpi_df.head()


,Rank,Country,Score,Change
159,160,Afghanistan,3.294,NaN
41,42,Albania,1.809,3.0
89,90,Algeria,2.110,2.0
71,72,Angola,2.043,19.0
46,47,Argentina,1.855,2.0


In [11]:
# rename column and drop other column
gpi_df = gpi_df.rename(columns={
                                'Country' : 'country',
                                'Score' : 'Global_Peace_Index_2023'
                                
                                })
gpi_df = gpi_df[['country', 'Global_Peace_Index_2023']]


gpi_df.head()

,country,Global_Peace_Index_2023
159,Afghanistan,3.294
41,Albania,1.809
89,Algeria,2.110
71,Angola,2.043
46,Argentina,1.855


#### for the steps below, refer to 2.1 above

In [12]:
# to identify the countries names that mismatch between IRCC and World bank

countries_wb = pd.DataFrame(countries_wb_df['country'].unique() , columns=['country'])
countries_gpi = pd.DataFrame(gpi_df['country'].unique(),  columns=['country'])

# Add a source column to indicate the origin
countries_wb['source'] = 'countries_wb'
countries_gpi['source'] = 'countries_gpi'

# Concatenate the two DataFrames
all_countries = pd.concat([countries_wb, countries_gpi], ignore_index=True)

# Find non-corresponding values
non_corresponding_values = all_countries.groupby('country').filter(lambda x: len(x) == 1)

# Display the non-corresponding values with their origin
print(non_corresponding_values)

# to change the countries name, printexport this DF below and work with the resul
# non_corresponding_values.to_csv('Output/non_corresponding_countries.csv')

                 country         source
0                  Aruba   countries_wb
4                Andorra   countries_wb
8         American Samoa   countries_wb
9    Antigua and Barbuda   countries_wb
20          Bahamas, The   countries_wb
..                   ...            ...
362           The Gambia  countries_gpi
366               Turkey  countries_gpi
375            Venezuela  countries_gpi
376              Vietnam  countries_gpi
377                Yemen  countries_gpi

[96 rows x 2 columns]


In [13]:
# Change countries name to match to th United Nations standard

gpi_df['country'] = gpi_df['country'].replace({
                            
                                                'Czech Republic' : 'Czechia' ,
                                                'Democratic Republic of the Congo' : 'Congo, Dem. Rep.' ,
                                                'East Timor' : 'Timor-Leste' ,
                                                'Egypt' : 'Egypt, Arab Rep.' ,
                                                'Iran' : 'Iran, Islamic Rep.' ,
                                                'Ivory Coast' : "Cote d'Ivoire" ,
                                                'Kyrgyzstan' : 'Kyrgyz Republic' ,
                                                'Laos' : 'Lao PDR' ,
                                                'North Korea' : "Korea, Dem. People's Rep." ,
                                                'Palestine' : 'West Bank and Gaza' ,
                                                'Republic of the Congo' : 'Congo, Rep.' ,
                                                'Russia' : 'Russian Federation' ,
                                                'Slovakia' : 'Slovak Republic' ,
                                                'South Korea' : 'Korea, Rep.' ,
                                                'Syria' : 'Syrian Arab Republic' ,
                                                'Taiwan' : 'Taiwan' ,
                                                'The Gambia' : 'Gambia, The' ,
                                                'Turkey' : 'Turkiye' ,
                                                'Venezuela' : 'Venezuela, RB' ,
                                                'Vietnam' : 'Viet Nam' ,
                                                'Yemen' : 'Yemen, Rep.' 

                                     })



In [14]:
# to identify the countries names that mismatch between IRCC and World bank

countries_wb = pd.DataFrame(countries_wb_df['country'].unique() , columns=['country'])
countries_gpi = pd.DataFrame(gpi_df['country'].unique(),  columns=['country'])

# Add a source column to indicate the origin
countries_wb['source'] = 'countries_wb'
countries_gpi['source'] = 'countries_gpi'

# Concatenate the two DataFrames
all_countries = pd.concat([countries_wb, countries_gpi], ignore_index=True)

# Find non-corresponding values
non_corresponding_values = all_countries.groupby('country').filter(lambda x: len(x) == 1)

# Display the non-corresponding values with their origin
print(non_corresponding_values)


                            country         source
0                             Aruba   countries_wb
4                           Andorra   countries_wb
8                    American Samoa   countries_wb
9               Antigua and Barbuda   countries_wb
20                     Bahamas, The   countries_wb
23                           Belize   countries_wb
24                          Bermuda   countries_wb
27                         Barbados   countries_wb
28                Brunei Darussalam   countries_wb
34                  Channel Islands   countries_wb
42                          Comoros   countries_wb
43                       Cabo Verde   countries_wb
46                          Curacao   countries_wb
47                   Cayman Islands   countries_wb
52                         Dominica   countries_wb
63                             Fiji   countries_wb
65                    Faroe Islands   countries_wb
66            Micronesia, Fed. Sts.   countries_wb
71                        Gibra

In [15]:
gpi_df.shape

(163, 2)

In [16]:
# Save to CSV 
gpi_df.to_csv("../Resources_Output/Global_Peace_Index.csv", index=False)

## 4) extract military index 

In [48]:
# URL of the webpage
url = "https://gmi.bicc.de/ranking-table"

# Send a request to fetch the content of the webpage
response = requests.get(url, verify=False)

# Check if the request was successful
if response.status_code == 200:
    # Parse the HTML content using BeautifulSoup
    soup = BeautifulSoup(response.content, "html.parser")
    
    # Find the table with the specified class
    table = soup.find("table", {"class": "table table-sm table-striped table-bordered table-data"})
    
    # Extract the table into a Pandas DataFrame
    if table:
        gmi_df = pd.read_html(str(table))[0]
        
        # Display the DataFrame
        gmi_df.head()

    else:
        print("Table not found on the webpage.")
else:
    print(f"Failed to retrieve webpage. Status code: {response.status_code}")

c:\Users\nazim\anaconda3\Lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'gmi.bicc.de'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
C:\Users\nazim\AppData\Local\Temp\ipykernel_12284\237859731.py:17: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  gmi_df = pd.read_html(str(table))[0]


In [49]:
gmi_df.head()

,Country,Military expenditure,Military personal,Heavy weapons,GMI score,GMI rank,GMI score trend
0,Ukraine,5.11,1.29,1.75,335,1,→ 81.31
1,Israel,2.15,1.77,3.06,257,2,→ -8.08
2,Armenia,2.00,1.79,2.28,223,3,→ -7.67
3,Qatar,2.99,0.77,2.23,220,4,-
4,Bahrain,1.92,0.65,2.68,215,5,→ -12.03


In [59]:
gmi_clean = gmi_df.copy()

In [60]:
# to identify the countries names that mismatch between IRCC and World bank

countries_wb = pd.DataFrame(countries_wb_df['country'].unique() , columns=['country'])
countries_gmi = pd.DataFrame(gmi_clean['Country'].unique(),  columns=['country'])

# Add a source column to indicate the origin
countries_wb['source'] = 'countries_wb'
countries_gmi['source'] = 'countries_gpi'

# Concatenate the two DataFrames
all_countries = pd.concat([countries_wb, countries_gmi], ignore_index=True)

# Find non-corresponding values
non_corresponding_values = all_countries.groupby('country').filter(lambda x: len(x) == 1)

# Display the non-corresponding values with their origin
print(non_corresponding_values)

# to change the countries name, printexport this DF below and work with the resul
non_corresponding_values.to_csv('non_corresponding_countries.csv')

                               country         source
0                                Aruba   countries_wb
4                              Andorra   countries_wb
5                 United Arab Emirates   countries_wb
8                       American Samoa   countries_wb
9                  Antigua and Barbuda   countries_wb
..                                 ...            ...
322                          Swaziland  countries_gpi
337                      Cote D'Ivoire  countries_gpi
342                             Gambia  countries_gpi
354  Congo, Democratic Republic of the  countries_gpi
360                         Cape Verde  countries_gpi

[102 rows x 2 columns]


In [61]:
# Change countries name to match to th United Nations standard

gmi_clean['Country'] = gmi_clean['Country'].replace({

                                        'Russia'    :    'Russian Federation'   ,
                                        'Korea, Republic of'    :    'Korea, Rep.'   ,
                                        'Brunei'    :    'Brunei Darussalam'   ,
                                        'United States of America'    :    'United States'   ,
                                        'Iran'    :    'Iran, Islamic Rep.'   ,
                                        'Egypt'    :    'Egypt, Arab Rep.'   ,
                                        'Turkey'    :    'Turkiye'   ,
                                        'Congo, Republic of'    :    'Congo, Rep.'   ,
                                        'Kyrgyzstan'    :    'Kyrgyz Republic'   ,
                                        'Slovakia'    :    'Slovak Republic'   ,
                                        'Venezuela'    :    'Venezuela, RB'   ,
                                        'Czech Republic'    :    'Czechia'   ,
                                        'Swaziland'    :    'Eswatini'   ,
                                        "Cote D'Ivoire"    :    "Cote d'Ivoire"   ,
                                        'Gambia'    :    'Gambia, The'   ,
                                        'Congo, Democratic Republic of the'    :    'Congo, Dem. Rep.'   ,
                                        'Cape Verde'    :    'Cabo Verde'   ,


                                     })

In [62]:
# to identify the countries names that mismatch between IRCC and World bank

countries_wb = pd.DataFrame(countries_wb_df['country'].unique() , columns=['country'])
countries_gmi = pd.DataFrame(gmi_clean['Country'].unique(),  columns=['country'])

# Add a source column to indicate the origin
countries_wb['source'] = 'countries_wb'
countries_gmi['source'] = 'countries_gpi'

# Concatenate the two DataFrames
all_countries = pd.concat([countries_wb, countries_gmi], ignore_index=True)

# Find non-corresponding values
non_corresponding_values = all_countries.groupby('country').filter(lambda x: len(x) == 1)

# Display the non-corresponding values with their origin
print(non_corresponding_values)



                  country        source
0                   Aruba  countries_wb
4                 Andorra  countries_wb
5    United Arab Emirates  countries_wb
8          American Samoa  countries_wb
9     Antigua and Barbuda  countries_wb
..                    ...           ...
209              Viet Nam  countries_wb
210               Vanuatu  countries_wb
211                 Samoa  countries_wb
212                Kosovo  countries_wb
213           Yemen, Rep.  countries_wb

[68 rows x 2 columns]


In [63]:
gmi_clean.columns

Index(['Country', 'Military expenditure', 'Military personal', 'Heavy weapons',
       'GMI score', 'GMI rank', 'GMI score trend'],
      dtype='object')

In [69]:
gmi_clean_df = gmi_df.rename(columns={
                                    'Country' : 'country'  ,
                                    'GMI score' : 'GMI_score'
                                    })
gmi_clean_df = gmi_clean_df.drop(columns=['GMI rank', 'GMI score trend'])
gmi_clean_df = gmi_clean_df.sort_values('country')
gmi_clean_df.head(5)

,country,Military expenditure,Military personal,Heavy weapons,GMI_score
110,Afghanistan,1.17,0.75,0.01,70
101,Albania,1.17,0.36,0.68,80
17,Algeria,2.31,0.83,1.70,178
49,Angola,1.20,0.93,1.20,122
131,Argentina,0.39,0.23,0.90,55


In [70]:
        
# Save the DataFrame to a CSV file
gmi_clean_df.to_csv("../Resources_Output/global_military_index.csv", index=False)
print("Data saved to ranking_table.csv")

Data saved to ranking_table.csv
